```
   ____         __  _                __       ________            __       
  / __/__ ___  / /_(_)_ _  ___ ___  / /____ _/ /_  __/______ ____/ /__ ____
 _\ \/ -_) _ \/ __/ /  ' \/ -_) _ \/ __/ _ `/ / / / / __/ _ `/ _  / -_) __/
/___/\__/_//_/\__/_/_/_/_/\__/_//_/\__/\_,_/_/ /_/ /_/  \_,_/\_,_/\__/_/   
``` 

## 1. Low-Asset trading bot using sentiment analysis 

Most quantitative strategies require initial investments of rather large (depending on the trader's capital) amounts, often in the order of 10 to 100 thousand dollars. This has a mathematical reason to it. Among these reasons, the transaction cost is the main culprit, as it is not scalable. Let's say the commission to pay for a trade is of 1\$. Starting from a capital of 100\$, a single transaction has a baseline cost of 1% of the trader's capital, while if the latter is, say, 100 times higher, the ratio drops to 0.01%. Thus, in the first case, a 1% gain s needed to break even, while in the second, a 0.01% gain is needed. Over time, the commission costs of all made trades can amount to the totality (or more) of the initial capital if the latter is too small. High-frequency trading is therefore virtually impossible.
Kelly's criterion, giving the optimal position size is: \
$optimal\ position = \frac{p \times b -q}{b}$ , with: 
- $p$ the probability of winning
- $q = 1 - p$ the probability of losing
- $b$ the win/loss ratio
  
The above formula says that, with a given strategy, the bad trade will represent a much higher loss, relatively to the account.
Furthemore, the above criterion assumes fractional positions, which are not available for small accounts, often forces to buy whole shares. This in turn breaks all hope for mathematical optimization.
The variance of returns for a given strategy also justifies large accounts compared to smaller ones, as the latter, by trading more positions, will smooth out the variance.
Multiple other mathematical reasons justify this claim. You can add to those discussed above the drawdown recovery, forced investments to high volatility positions for small cap accounts, the Sharpe ration degradation, compound effects, etc.

By combining all of them, we can derive a minimum account size formula defining the minimum viable account size: \
$min(size) = \frac{c_{fixed} \times n_{trades} \times 100}{c_{acceptable}}$ , with:
- $c_{fixed}$ the fixed cost per trade
- $n_{trades}$ the number of trades per year
- $c_{acceptable}$ the acceptable (position?) cost

### "So what should I do?"

Among the identified reasons why large accounts are better than small accounts, the strategy to adopt if you still want to trade with small sums of money is:
- fewer trades to dodge stacking transaction costs
- invest in fractional shares or cryptocurrencies to avoid discrete sizing
- have a higher edge per trade to mitigate transaction costs best
- boost win rate, for instance using sentiment and news analysis, this allows to mix the reactive nature of traditional trading indicators, which are based on past data, with more predictive analysis.

An mixed-strategy would unfold into 3 phases: 
#### Phase 1: Capital Building
Build up capital by **investing cryptocurrencies with a conservative approach**, try achieving a **150% to 200% return**. 
#### Phase 2: Transition
Once this is reached, maybe over the course of 3 to 6 months, **transition to 50% crypto and 50% fractional shares**, allowing a diversification of positions. 
#### Phase 3: Scale Up
Once the account holds 5000$ or more, more traditional quantitative tradin gcan take place.

Now let's implement that.

### "OK, but sentiment analysis is **really** outdated, so how can my bot do better?"

Well, spot on. It is common knowledge that platforms like Twitter and Reddit host an ever (scaringly) growing number of bots and actors that manipulate the sentiment associated with a given (crypto) asset, often to the detriment of naive investors. Multiple things can be done to mitigate the negative effects of the market manipulation, but the most reliable one would be to refrain from using them and opt for traditional news platforms. Problem is, most of them don't actually provide free APIs in the way we wish to use them. 

## 2. Implementation

Here's a diagram of each phase of the trading bot's decisions: 
```
1. TEMPORAL CHECK
   ├─ Check signal age for existing positions
   ├─ If signals decayed → Force reassessment
   └─ Collect fresh data

2. DATA COLLECTION
   ├─ Technical indicators
   ├─ News sentiment
   ├─ Market structure
   └─ Generate new signals with timestamps

3. ADVERSARIAL ANALYSIS
   ├─ Bull LLM: Make case for buying
   ├─ Bear LLM: Counter with sell case
   └─ Judge LLM: Evaluate both arguments
   
4. RECURSIVE VALIDATION
   ├─ Critic LLM: Challenge the judge's decision
   ├─ If flaws found → Revise decision
   ├─ Challenge revised decision
   └─ Repeat until validated or depth reached

5. EXECUTION
   ├─ Check confidence threshold (≥0.5)
   ├─ Calculate position size
   ├─ Execute trade
   └─ Log everything for analysis
```


In [2]:
import yfinance as yf
#from newsapi import NewsApiClient
#from tiingo import TiiingoClient 
import pandas as pd
import numpy as np
import ta
from datetime import datetime, timedelta
import time
import json
import requests
import logging
from textblob import TextBlob
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, asdict
from collections import defaultdict
import os

/home/diomir0/miniconda3/envs/quantfin/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
@dataclass 
class Signal:
    '''Represents a trading signal with temporal properties'''
    signal_type: str    # 'technical', 'news', 'market_structure'
    direction: str      # 'bullish', 'bearish', 'neutral'
    strength: float     # 0.0 to 1.0
    timestamp: datetime
    details: Dict

    def get_decayed_strength(self, halflife_hours: float = 6.0) -> float:
        '''Calculates the strength of the signal considering temporal decay'''
        age_hours = (datetime.now() - self.timestamp).total_seconds() / 3600
        decay_factor = 0.5 ** (age_hours / halflife_hours)     #TO-DO: The decay_factor could be optimized 
        return self.strength * decay_factor

In [4]:
class TemporalSignalTracker:
    '''
    Tracks all trading signals with temporal decay
    Enables reassessment when old signals lose relevance 
    '''
    def __init__(self, halflife_hours: float = 6.0, max_age_hours: float = 72.0):
        self.signals_by_symbol = defaultdict(list)
        self.halflife_hours = halflife_hours
        self.max_age_hours = max_age_hours

    def add_signal(self, symbol: str, signal: Signal):
        '''Add new signal and clean up old ones'''
        self.signals_by_symbol[symbol].append(signal)
        self._cleanup_old_signals(symbol)

    def _cleanup_old_signals(self, symbol: str):
        '''Remove signals that are older than max_age_hours'''
        cutoff = datetime.now() - timedelta(hours=self.max_age_hours)
        self.signals_by_symbol[symbol] = [
            s for s in self.signals_by_symbol[symbol]
            if s.timestamp > cutoff
        ]

    def get_current_confidence(self, symbol: str) -> Dict:
        '''
        Calculates current confidence considering temporal decay
        Returns breakdown by signal type
        '''
        signals = self.signals_by_symbol[symbol]

        if not signals:
            return {
                'total': 0.0,
                'technical': 0.0, 
                'news': 0.0,
                'market': 0.0,
                'signal_count': 0, 
                'oldest_signal_age_hours': 0
            }
        # Calculate decayed strengths by type
        by_type = defaultdict(list)
        for signal in signals:
            decayed = signal.get_decayed_strength(self.halflife_hours)
            by_type[signal.signal_type].append(decayed)

        # Aggregate
        confidence = {
            'technical': np.mean(by_type['technical']) if by_type['technical'] else 0.0,
            'news': np.mean(by_type['news']) if by_type['news'] else 0.0,
            'market': np.mean(by_type['market_structure']) if by_type['market_structure'] else 0.0,
            'signal_count': len(signals),
            'oldest_signal_age_hours': (datetime.now() - min(s.timestamp for s in signals)).total_seconds / 3600
        }

        # Total confidence is a weighted average - TO-DO: Optimize weights
        confidence['total'] = (
            confidence['technical'] * 0.5 +
            confidence['news'] * 0.3 +
            confidence['market'] *0.2
        )
        
        return confidence

    def should_reassess(self, symbol: str, threshold: float = 0.3) -> Tuple[bool, str]:
        '''
        Determines if the position should be reassessed due to signal decay
        Returns (should_reassess, reason)
        '''
        confidence = self.get_current_confidence(symbol)

        if confidence['total'] < threshold:
            return True, f"Signal confidence decayed to {confidence['total']:.2f} (threshold: {threshold})"

        if confidence['oldest_signal_age_hours'] > 48:
            return True, f"Oldest signal is {confidence['odlest_signal_age_hours']:.1f} hours old (reassessment at 48 hours)"

        return False, "Signal still fresh"

    def get_signal_summary(self, symbol: str) -> str:
        '''Generate human-readable summary of tcurretn signals'''
        confidence = self.get_current_confidence(symbol)
        signals = self.signals_by_symbol[symbol]

        if not signals:
            return "No active signals"

        summary = f'''Signal status for {symbol}:
        - Total confidence: {confidence['total']:.2f}
        - Technical: {confidence['technical']:.2f}
        - News: {confidence['news']:.2f}
        - Market: {confidence['market_structure']:.2f}
        - Active signals: {confidence['signal_count']:.1f}
        - Oldest signal: {confidence['oldest_signal_age_hours']:.1f}h ago 
        '''
        return summary

In [9]:
class LLMTradingBot:
    '''
    Trading bot with:
    - Adversarial reasoning (bull vs. bear debate)
    - Recursive self-validation 
    - Temporal signal decay
    '''

    def __init__(self, initial_balance=400, llm_endpoint="http://localhost:11434"):
        self.balance = initial_balance
        self.positions = {}
        self.trade_history = []
        self.decision_log = []

        # LLM configuration
        self.llm_endpoint = llm_endpoint
        self.llm_model = "dolphin3:8b"

        # Trading parameters 
        self.symbols = ['BTC-USD', 'ETH-USD', 'SOL-USD']
        self.max_position_size = 0.45
        self.stop_loss_pct = 0.04
        self.take_profit_pct = 0.10
        self.min_trade_size = 10

        # Temporal decay system
        self.signal_tracker = TemporalSignalTracker(
            halflife_hours=6.0, 
            max_age_hours=12.0
        )

        # Adversarial parameters
        self.enable_adversarial = True
        self.recursive_depth = 2        # Number of times decisions are challenged

        # Performance tracking
        self.reassessment_triggers = []
        
        logger.info(f"SentimentalTrader initialized")
        logger.info(f"Features: Adversarial={self.enable_adversarial},"
                    f"Recursive depth={self.recursive_depth},"
                    f"Temporal decay half-life={self.signal_tracker.halflife_hours}h")
        self._verify_llm_connection()

    def _verify_llm_connection(self):
        '''Verify LLM is accessible'''
        try:
            response = requests.get(f"{self.llm_endpoint}/api/tags", timeout=5)
            if response.status_code == 200:
                logger.info("✓ LLM connected")
            else:
                logger.info("LLM not responding")
        except Excption as e:
            logger.error(f"Cannot connect to LLM: {e}")

    def query_llm(self, prompt: str, temperature: float = 0.1, role: str = "neutral") -> str:
        '''Query local LLM with role context'''
        try:
            # Add role context to prompt
            role_contexts = {
                'bull': "/set system You are an AGGRESSIVELY BULLISH trader who looks for reasons to buy.",
                'bear': "/set system You are an EXTREMELY BEARISH trader who looks for reasons to sell.",
                'judge': "/set system You are an IMPARTIAL JUDGE who weighs evidence objectively.",
                'critic': "/set system You are a CRITICAL ANALYST who finds flaws in reasoning.",
                'neutral': ""
            }

            full_prompt = role_contexts.get(role, "") + "\n\n" + prompt

            payload = {
                "model": self.llm_model,
                "prompt": full_prompt,
                "stream": False, 
                "options": {
                    "temperature": temperature,
                    "top_p": 0.9,
                    "max_tokens": 600
                }
            }

            response = requests.post(
                f"{self.llm_endpoint}/api/generate",
                json=payload,
                timeout=45
            )

            if response.status_code == 200:
                return response.json().get('response', '').strip()
            else:
                logger.error(f"LLM error: {response.status_code}")
                return ""

        except Exception as e:
            logger.error(f"LLM query failed: {e}")
            return ""

    #=============== DATA COLLECTION ===============#

    def _collect_technical(self, symbol: str) -> Dict:
        '''Collect technical data'''
        try:
            df = yf.download(symbol, period="3mo", interval="1h", progress=False)
    
            if len(df) < 50:
                return {}
    
            # Calculate indicators
            df['SMA_7'] = ta.trend.sma_indicator(df['Close'], window=7)
            df['SMA_21'] = ta.trend.sma_indicator(df['Close'], window=21)
            df['RSI'] = ta.trend.rsi(df['Close'], window=14)
    
            macd = ta.trend.MACD(df['Close'])
            df['MACD'] = macd.macd()
            df['MACD_signal'] = macd.macd_signal()
    
            latest = df.iloc[-1]
            prev = df.iloc[-2]
    
            return {
                'price': float(latest['Close']),
                'price_change_24h': float(latest['Close'] - df.iloc[-24]['Close']),
                'rsi': float(latest['RSI']),
                'sma_7': float(latest['SMA_7']),
                'sma_21': float(latest['SMA_21']),
                'macd': float(latest['MACD']),
                'macd_signal': float(latest['MACD_signal']),
                'trend': 'uptrend' if latest['SMA_7'] > latest['SMA_21'] else 'downtrend',
                'macd_crossover': 'bullish' if prev['MACD'] < prev['MACD_signal'] and latest['MACD'] > latest['MACD_signal'] else "bearish" if prev['MACD'] > prev['MACD_signal'] and latest['MACD'] < latest['MACD_signal'] else "none"
            }
    
        except Exception as e:
            logging.error(f"Technical data error: {e}")
            return {}

    def _collect_news(self, symbol: str) -> Dict:
        '''Collect news sentiment'''
        # Yahoo! finances API
        try:
            ticker = yf.Ticker(symbol)
            yf_news = ticker.news
    
            if not yf_news:
                return {'sentiment': 0.0, 'count': 0, 'headlines': []}
    
            yf_sentiments = []
            yf_headlines = []
    
            for item in yf_news[:10]:
                title = item.get('title', '')
                if len(title) > 20: 
                    blob = TextBlob(title)
                    yf_sentiments.append(blob.sentiment.polarity)
                    yf_headlines.append(title)
    
            yf_avg_sentiment = np.mean(yf_sentiments) if yf_sentiments else 0.0
    
            return {
                'api': "yf",
                'sentiment': float(yf_avg_sentiment),
                'count': len(yf_headlines),
                'headlines': yf_headlines[:3]
            }
    
        except Exception as e:
            logger.error(f"YF News error: {e}")
            return {}
    
        '''
        # NewsAPI
        try: 
            newsapi = NewsApiClient(api_key=os.environ["NEWSAPI_KEY"])
            newsapi_news = newsapi.get_top_headlines(q='bitcoin',
                                                     sources='bbc-news,bloomberg,crypto-coins-news,financial-post,the-wall-street-journal,fortune',
                                                     category='business',
                                                     language='en',
                                                     country='us')  
            if not newsapi_news:
                return {'sentiment': 0.0, 'count': 0, 'headlines': []}
    
            newsapi_sentiments = []
            newsapi_headlines = []
    
            for item in newsapi_news[:10]:
                title = item.get('title', '')
                if len(title) > 20: 
                    blob = TextBlob(title)
                    newsapi_sentiments.append(blob.sentiment.polarity)
                    newsapi_headlines.append(title)
    
            newsapi_avg_sentiment = np.mean(newsapi_sentiments) if newsapi_sentiments else 0.0
    
            return {
                'api': "yf"
                'sentiment': float(newsapi_avg_sentiment),
                'count': len(newsapi_headlines)
                'headlines': newsapi_headlines[:3]
            }
    
        except Exception as e:
            logger.error(f"NewsAPI News error: {e}")
            return {}
        '''

    def _collect_market_structure(self, symbol: str) -> Dict:
        '''Collect market structure'''
        try: 
            df = yf.download(symbol, period="7d", interval="1h", progress=False)
    
            if len(df) < 24:
                return {}
    
            current = float(df['Close'].iloc[-1])
            resistance = float(df['High'].tail(168).nlargest(5).mean())
            support = float(df['Low'].tail(168).nsmallest(5).mean())
    
            return {
                'support': support,
                'resistance': resistance,
                'distance_to_resistance': ((resistance - current) / current * 100),
                'distance_to_support': ((current - support) / current * 100)
            }
    
        except Exception as e:
            logger.error(f"Market structure error: {e}")
            return {}

    #=============== SIGNAL GENERATION ===============#
    
    def _generate_signals_from_data(self, symbol: str, data: Dict):
        '''Generate and store signals from collected data'''
        timestamp = data['timestamp']
    
        # Technical signals
        if data['technical']:
            tech = data['technical']
    
            # RSI signal
            if tech['rsi'] < 30:
                self.signal_tracker.add_signal(symbol, Signal(
                    signal_type='technical',
                    direction='bullish',
                    strength=0.8,
                    timestamp=timestamp,
                    details={'indicator': 'RSI', 'value': tech['rsi']}
                )) 
            elif tech['rsi'] > 70:
                self.signal_tracker.add_signal(symbol, Signal(
                    signal_type='technical',
                    direction='bearish',
                    strength=0.8,
                    timestamp=timestamp,
                    details={'indicator': 'RSI', 'value': tech['rsi']}
                ))
    
            # MACD signal
            if tech['macd_crossover'] == 'bullish':
                self.signal_tracker.add_signal(symbol, Signal(
                    signal_type='technical',
                    direction='bullish',
                    strength=0.8,
                    timestamp=timestamp,
                    details={'indicator': 'MACD', 'crossover': 'bullish'}
                )) 
            elif tech['macd_crossover'] == 'bearish':
                self.signal_tracker.add_signal(symbol, Signal(
                    signal_type='technical',
                    direction='bearish',
                    strength=0.8,
                    timestamp=timestamp,
                    details={'indicator': 'MACD', ' crossover': 'bearish'}
                ))
    
        # News signal
        if data['news'] and data['news']['count'] > 0:
            sentiment = data['news']['sentiment']
            strength = min(abs(sentiment), 1.0)
            direction = 'bullish' if sentiment > 0 else 'bearish' if sentiment < 0 else 'neutral'
    
            self.signal_tracker.add_signal(symbol, Signal(
                signal_type='news',
                direction=direction,
                strength=strength,
                timestamp=timestamp,
                details={'sentiment_score': sentiment, 'article_count': data['news']['count']}
            ))
    
        # Market structure signal
        if data['market']:
            market = data['market']
            if market['distance_to_support'] < 3:
                self.signal_tracker.add_signal(symbol, Signal(
                    signal_type='market_structure',
                    direction='bullish',
                    strength=0.6,
                    timestamp=timestamp,
                    details={'reason': 'near_support'}
                ))
            elif market['distance_to_resistance'] < 3:
                self.signal_tracker.add_signal(symbol, Signal(
                    signal_type='market_structure',
                    direction='bearish',
                    strength=0.6,
                    timestamp=timestamp,
                    details={'reason': 'near_resistance'}
                ))

    #=============== ADVERSARIAL REASONING  ===============#

    def adversarial_analysis(self, symbol: str, data: Dict) -> Dict:
        '''
        Bull vs Bear, judged by impartial LLM, if that exists
        '''
        logger.info(f"Starting adversarial analysis for {symbol}...")

        # Get temporal signal status
        signal_summary = self.signal_tracker.get_signal_summary(symbol)

        # Create data summary for LLMs
        data_summary = self._format_data_for_llm(symbol, data, signal_summary)

        # Step 1: Bull case
        logger.info("Querying bull LLM...")
        bull_prompt = f"""You are an AGGRESSIVELY BULLISH trader analyzing {symbol}.

        {data_summary}

        Make the STRONGEST possible case for BUYING. Find every bullish signal. Be agressive but honest, and DO NOT OVERESTIMATE the target confidence. Format your argument as:

        BULL CASE:
        - Key bullish factors: [list 3-5]
        - Why bearish factors are overblown: [address concerns]
        - Target confidence: [0.0-1.0]
        - Recommend position size: [percentage]
        """

        bull_case = self.query_llm(bull_prompt, temperature=0.2, role='bull')

        # Step 2: Bear case
        logger.info("Querying for bear LLM...")
        bear_prompt = f"""You are an EXTREMELY BEARISH trader analyzing {symbol}.

        {data_summary}

        The bull trader argued:
        {bull_case}

        Make the STRONGEST possible case for SELLING or AVOIDING. Find every bearish signal and counter the bull's arguments. Be pessimistic but honest, and DO NOT OVERESTIMATE the risk. Format your argument as:

        BEAR CASE:
        - Key bearish factors: [list 3-5]
        - Why bullish factors are overblown: [counter bull's points]
        - Risk assessment: [What could go wrong]
        - Recommended action: [sell/avoid]
        """

        bear_case = self.query_llm(bear_prompt, temperature=0.2, role='bear')

        # Step 3: Judge evaluates both 
        logger.info("Querying for the judge LLM...")
        judge_prompt = f"""You are an IMPARTIAL JUDGE evaluating a trading debate for {symbol}.

        {data_summary}

        BULL ARGUMENT:
        {bull_case}

        BEAR ARGUMENT:
        {bear_case}

        Evaluate both arguments obkectively. Consider:
        1. Which side has more verifiable facts vs. speculation?
        2. Are the risks real or exaggerated?
        3. Are the opportunities real or overhyped?
        4. Given the account size (${self.balance:.2f}), what's the right decision?

        Provide your judgement in JSON format:
        {{
            "decision": "buy"|"sell"|"hold",
            "confidence": 0.0-1.0,
            "position_size_pct": 0-100,
            "reasoning": "explain which argument won and why",
            "bull_score": 0-10,
            "bear_score": 0-10,
            "key_deciding_factor": ["factor1", "factor2"]
        }}
        """

        judge_response = self.query_llm(judge_prompt, temperature=0.1, role='judge')

        # Parse judge's decision
        try:
            json_start = judge_response.find('{')
            json_end = judge_response.find('}') + 1
            judge_decision = json.loads(judge_response[json_start:json_end])

            # Add metadata
            judge_decision['bull_case'] = bull_case[:200]
            judge_decision['bear_case'] = bear_case[:200]
            judge_decision['adversarial_used'] = True

            return judge_decision

        except Exception as e:
            logger.error(f"Failed to parse judge decision: {e}")
            return None

    #=============== RECURSIVE VALIDATION  ===============#

    def recursive_validation(self, symbol: str, initial_decision: Dict, depth: int = 2) -> Dict:
        """
        Recursively challenge and validate the decision
        """
        logger.info(f"Starting recursive validation (depth={depth})...")

        current_decision = initial_decision
        validation_history = []

        for level in range(depth):
            logger.info(f"Validation level {level + 1}/{depth}...")

            critique_prompt = f"""You are a CRITICAL ANALYST reviewing a trading decision for {symbol}.

            PROPOSED DECISION:
            - Action: {current_decision['decision']}
            - Confidence: {current_decision['confidence']}
            - Position size: {current_decision['position_size_pct', 0]}%
            - Reasoning: {current_decision['reasoning']}
            
            CHALLENGE THIS DECISION:
            1. What's the WEAKEST part of this reasoning?
            2. What evidence CONTRADICTS this decision?
            3. What risks are being UNDERESTIMATED?
            4. If this decision loses money, why would it be?
            5. What would a more experienced trader critique?

            After this critique, respond in JSON:
            {{
                "verdict": "validated"|"needs_revision"|"reverse_decision",
                "critique": "detailed critisism",
                "revised_decision": "buy"|"sell"|"hold" (if needs revision),
                "revised confidence": 0.0-1.0 (if needs revision),
                "revised_reasoning": "explanantion" (if needs revision)
            }}
            """

            critique_response = self.query_llm(critique_prompt, temperature=0.1, role='critic')
            
            try:
                json_start = critique_reponse.find('{')
                json_end = critique_reponse.find('}') + 1
                critique = json.loads(critique_response[json_start:json_end])

                validation_history.append({
                    'level': level + 1,
                    'verdict': critique_response['verdict'],
                    'critique': critique_response['critique']
                })

                if critique['verdict'] == 'validated':
                    logger.info(f"✓ Decision validated at level {level + 1}")
                    break
                elif critique['verdict'] in ['needs_revision', 'reverse_decision']:
                    logger.info(f"⚠ Decision revised at level {level + 1}")
                    current_decision['decision'] = critique.get('revised_decision', current_decision['decision'])
                    current_decision['confidence'] = critique.get('revised_confidence', current_decision['confidence'])
                    current_decision['reasoning'] = critique.get('revised_reasoning', current_decision['reasoning'])

            except Exception as e:
                logger.error(f"Validation level {level + 1} failed: {e}")
                break

        current_decision['validation_history'] = vaidation_history
        current_decision['validation_depth'] = len(validation_history)

        return current_decision

    #=============== MAIN DECISION PIPELINE  ===============#

    def get_comprehensive_decision(self, symbol: str) -> Optional[Dict]:
        """
        Complete decision pipeline:
        1. Check temporal decay (should we reassess?)
        2. Collect fresh data
        3. Adversarial analysis
        4. Recursive validation
        5. Final decision
        """
        logger.info(f"\n{'='*70}")
        logger.info(f"COMPREHENSIVE ANALYSIS FOR {symbol}")
        logger.info(f"{'='*70}")

        # Step 1: Check if we need to reassess existing position
        if symbol in self.positions:
            should_reassess, reason = self.signal_tracker.should_reassess(symbol)
            if should_reassess:
                logger.warning(f"⚠ Position reassessment triggered: {reason}")
                self.reassessment_triggers.append({
                    'symbol': symbol,
                    'timestamp': datetime.now(),
                    'reason': reason
                })
                
        # Step 2: Collect all data
        data = self.collect_all_data(symbol)

        if not data['technical']:
            logger.error(f"Cannot analyze {symbol} - insufficient data")
            return None

        # Step 3: Adversarial analysis
        if self.enable_adversarial:
            decision = self.adversarial_analysis(symbol, data)
        else:
            decision = self._simple_analysis(symbol, data)

        if not decision:
            logger.error(f"Failed to generate decision for {symbol}")
            return None

        # Step 4: Recursive validation
        if self.recursive_depth > 0:
            decision = self.recursive_validation(symbol, decision, self.recursive_depth)

        # Step 5: Add metadata
        decision['symbol'] = symbol
        decision['timestamp'] = datetime.now().isoformat()
        decision['temporal_confidence'] = self.signal_tracker.get_current_confidence(symbol)
        decision['data'] = data

        # Log decision
        self.decision_log.append(decision)

        logger.info(f"\n{'='*70}")
        logger.info(f"FINAL DECISION FOR {symbol}")
        logger.info(f"{'='*70}")
        logger.info(f"Action: {decision['decision'].upper}")
        logger.info(f"Confidence: {decision['confidence']:.2f}")
        logger.info(f"Position Size: {decision.get('position_size_pct', 0)}%")
        logger.info(f"Reasoning: {decision['reasoning'][:150]}...")
        if decision.get('validation_history'):
            logger.info(f"Validation: Passed {len(decision.get('validation_history'))} levels")
        logger.info(f"{'='*70}\n")

        return decision

    def _simple_analysis(self, symbol: str, data: Dict) -> Dict:
        """Fallback simple analysis if adversarial is disabled"""
        return {
            'decision': 'hold',
            'confidence': 0.5,
            'position_size_pct': 0,
            'reasoning': 'Simple ananysis - adversarial disbaled'
        }

    def _format_data_for_llm(self, symbol: str, data: Dict, signal_summary: str) -> str:
        """Format collected data for LLM consumption"""
        tech = data.get('technical', {})
        news = data.get('news', {})
        market = data.get('market', {})

        # Position info
        position_info = "No current position"
        if symbol in self.positions:
            pos = self.positions[symbol]
            position_info = f"Current position: {pos['shares']:.4f} shares at ${pos['avg_price']:.2f}"

            return f"""
            SYMBOL: {symbol}
            ACCOUNT BALANCE: ${self.balance:.2f}
            {position_info}

            TEMPORAL SIGNAL STATUS:
            {signal_summary}

            TECHNICAL DATA:
            - PRICE: ${tech.get('price', 0):.2f}
            - 24h Change: {tech.get('price_change_24h', 0):+.2f}%
            - RSI: {tech.get('rsi', 50):.1f}
            - Trend: {tech.get('trend', 'unknown')}
            - MACD: {tech.get('macd_crossover', 'none')}

            NEWS SENTIMENT:
            - Sentiment: {news.get('sentiment', 0):+.2f}
            - Articles: {news.get('count', 0)}
            - Headlines: {'; '.join(news.get('headlines', [])[:2])}

            MARKET STRUCTURE:
            - Support: ${marker.get('support', 0):.2f}
            - Resistance: ${market.get('resistance', 0):.2f}
            - Distance to resistance: {market.get('distance_to_resistance', 0):.1f}%
            - Distance to support: {market.get('distance_to_support', 0):.1f}%
            """

    #=============== EXECUTION  ===============#

    def execute_decision(self, decision: Dict) -> bool:
        """Execute the final decision"""
        symbol = decision['symbol']
        action = decision['decision']
        confidence = decision['confidence']

        # Confidence threshold
        if confidence < 0.5:
            logger.info(f"Skipping {action} - confidence too low ({confidence:.2f})")
            return False

        current_price = decision['data']['technical']['price']

        if action == 'buy':
            if symbol in self.positions:
                logger.info(f"Already holding {symbol}")
                return False

            # Calculate position size
            max_investment = self.balance * self.max_position_size
            actual_investment = max_investment * (decision.get('position_size_pct', 50) / 100)
            actual_investment = min(actual_investment, self.balance * 0.95)

            if actual_investment < self.min_trade_size:
                logger.info(f"Investment too small: ${actual_investment:.2f}")
                return False
    
            shares = actual_investment / current_price
            self.balance -= actual_investment
            self.positions[symbol] = {
                'shares': shares,
                'avg_price': current_price,
                'entry_time': datetime.now(),
                'entry_reasoning': action['reasoning'][:200]
            }
    
            self.trade_history.append({
                'timestamp': datetime.now(),
                'symbol': symbol,
                'action': 'buy',
                'shares': shares,
                'price': current_price,
                'total': actual_investment,
                'confidence': confidence,
                'adversarial': decision.get('adversarial_used', False),
                'validated': decision.get('validation_depth', 0) > 0
            })
    
            logger.info(f"✓ EXECUTED BUY: {shares:.4f} {symbol} @ ${current_price:.2f}")
            return True

        elif action == 'sell':
            if symbol not in self.positions:
                logger.info(f"No position in {symbol} to sell")
                return False

        pos = self.positions[symbol]
        shares = pos['shares']
        revenue = shares * current_price
        cost = shares * pos['avg_price']
        profit = revenue - cost
        profit_pct = (profit / cost * 100)

        self.balance += revenue
        del self.positions[symbol]

        # Clear old signals after selling
        self.signal_tracker.signals_by_symbol[symbol] = []

        self.trade_history.append({
            'timestamp': datetime.now(),
            'symbol': symbol,
            'action': 'sell',
            'shares': shares,
            'price': current_price,
            'total': revenue,
            'profit': profit,
            'profit_pct': profit_pct,
            'confidence': confidence,
            'adversarial': decision.get('adversarial_used', False)
            'validated': decison.get('validation_depth', 0) > 0
        })

        logger.info(f"✓ EXECUTED SELL: {shares:.4f} {symbol} @ ${current_price:.2f}")
            
